In [1]:
!!pip install torch transformers tensorly

['Requirement already satisfied: torch in /usr/local/lib/python3.10/dist-packages (2.5.1+cu121)',
 'Requirement already satisfied: transformers in /usr/local/lib/python3.10/dist-packages (4.47.0)',
 'Collecting tensorly',
 '  Downloading tensorly-0.9.0-py3-none-any.whl.metadata (8.6 kB)',
 'Requirement already satisfied: filelock in /usr/local/lib/python3.10/dist-packages (from torch) (3.17.0)',
 'Requirement already satisfied: typing-extensions>=4.8.0 in /usr/local/lib/python3.10/dist-packages (from torch) (4.12.2)',
 'Requirement already satisfied: networkx in /usr/local/lib/python3.10/dist-packages (from torch) (3.4.2)',
 'Requirement already satisfied: jinja2 in /usr/local/lib/python3.10/dist-packages (from torch) (3.1.4)',
 'Requirement already satisfied: fsspec in /usr/local/lib/python3.10/dist-packages (from torch) (2024.12.0)',
 'Requirement already satisfied: sympy==1.13.1 in /usr/local/lib/python3.10/dist-packages (from torch) (1.13.1)',
 'Requirement already satisfied: mpmat

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
model_name = "cerebras/Cerebras-GPT-111M"  # Choose the model size
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move the model to GPU (if available)
model.to("cuda")

config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/486M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-9): 10 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): GELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("cerebras/Cerebras-GPT-6.7B")

# Set the padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

# Load a dataset (e.g., wikitext for language modeling)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenize the dataset
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    tokenized["labels"] = tokenized["input_ids"]  # Add labels
    return tokenized

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set format for PyTorch (include labels)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Verify the dataset
print(tokenized_dataset["train"][0].keys())

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [5]:
from torch.utils.data import DataLoader

# Create dataloaders
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=8, shuffle=True)
val_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=8)
test_dataloader = DataLoader(tokenized_dataset["test"], batch_size=8)

In [6]:
!pip install tdqm

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1320 sha256=ec47ccb8698ada0e456adf47a36a811ad19c8ecd494975c175acaeae6ccd17fe
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [7]:
import time
import psutil
import torch
from tqdm import tqdm  # Import tqdm

# Define the device (GPU or CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Function to measure energy and time
def measure_energy_and_time(model, dataloader, mode="train"):
    start_time = time.time()
    cpu_start = psutil.cpu_percent(interval=None)
    gpu_start = torch.cuda.memory_allocated() if device == "cuda" else 0

    if mode == "train":
        model.train()
        optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
        for batch in tqdm(dataloader, desc="Training", unit="batch"):  # Add tqdm
            inputs = batch["input_ids"].to(device)
            outputs = model(inputs, labels=inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    elif mode == "validate" or mode == "test":
        model.eval()
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Validation" if mode == "validate" else "Testing", unit="batch"):  # Add tqdm
                inputs = batch["input_ids"].to(device)
                outputs = model(inputs, labels=inputs)
                loss = outputs.loss

    end_time = time.time()
    cpu_end = psutil.cpu_percent(interval=None)
    gpu_end = torch.cuda.memory_allocated() if device == "cuda" else 0

    # Calculate metrics
    elapsed_time = end_time - start_time
    cpu_usage = cpu_end - cpu_start
    gpu_usage = (gpu_end - gpu_start) / (1024 ** 2)  # Convert to MB

    return elapsed_time, cpu_usage, gpu_usage

# Benchmark training
train_time, train_cpu, train_gpu = measure_energy_and_time(model, train_dataloader, mode="train")
print(f"Training Time: {train_time:.2f} seconds, CPU Usage: {train_cpu}%, GPU Usage: {train_gpu:.2f} MB")

# Benchmark validation
val_time, val_cpu, val_gpu = measure_energy_and_time(model, val_dataloader, mode="validate")
print(f"Validation Time: {val_time:.2f} seconds, CPU Usage: {val_cpu}%, GPU Usage: {val_gpu:.2f} MB")

# Benchmark testing
test_time, test_cpu, test_gpu = measure_energy_and_time(model, test_dataloader, mode="test")
print(f"Testing Time: {test_time:.2f} seconds, CPU Usage: {test_cpu}%, GPU Usage: {test_gpu:.2f} MB")

Training: 100%|██████████| 4590/4590 [35:21<00:00,  2.16batch/s]


Training Time: 2121.29 seconds, CPU Usage: 1.2000000000000028%, GPU Usage: 1738.97 MB


Validation: 100%|██████████| 470/470 [01:05<00:00,  7.17batch/s]


Validation Time: 65.60 seconds, CPU Usage: -72.9%, GPU Usage: 1146.03 MB


Testing: 100%|██████████| 545/545 [01:15<00:00,  7.19batch/s]

Testing Time: 75.81 seconds, CPU Usage: 27.0%, GPU Usage: 858.97 MB


In [8]:
# Prepare a single batch for inference
batch = next(iter(test_dataloader))
inputs = batch["input_ids"].to(device)

# Measure inference
start_time = time.time()
cpu_start = psutil.cpu_percent(interval=None)
gpu_start = torch.cuda.memory_allocated() if device == "cuda" else 0

with torch.no_grad():
    outputs = model(inputs)

end_time = time.time()
cpu_end = psutil.cpu_percent(interval=None)
gpu_end = torch.cuda.memory_allocated() if device == "cuda" else 0

# Calculate metrics
inference_time = end_time - start_time
cpu_usage = cpu_end - cpu_start
gpu_usage = (gpu_end - gpu_start) / (1024 ** 2)  # Convert to MB

print(f"Inference Time: {inference_time:.2f} seconds, CPU Usage: {cpu_usage}%, GPU Usage: {gpu_usage:.2f} MB")

Inference Time: 0.01 seconds, CPU Usage: -28.6%, GPU Usage: 1146.00 MB


In [9]:
import pandas as pd

results = {
    "Training": {"Time": train_time, "CPU Usage": train_cpu, "GPU Usage": train_gpu},
    "Validation": {"Time": val_time, "CPU Usage": val_cpu, "GPU Usage": val_gpu},
    "Testing": {"Time": test_time, "CPU Usage": test_cpu, "GPU Usage": test_gpu},
    "Inference": {"Time": inference_time, "CPU Usage": cpu_usage, "GPU Usage": gpu_usage},
}

df = pd.DataFrame(results)
print(df)

              Training   Validation     Testing    Inference
Time       2121.291219    65.595345   75.806017     0.005796
CPU Usage     1.200000   -72.900000   27.000000   -28.600000
GPU Usage  1738.971191  1146.031738  858.973145  1146.000000


In [10]:
def calculate_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating Accuracy", unit="batch"):
            inputs = batch["input_ids"].to(device)
            labels = inputs  # For next-token prediction, labels are the same as inputs
            outputs = model(inputs)
            predictions = outputs.logits.argmax(dim=-1)  # Get predicted token IDs
            correct += (predictions == labels).sum().item()
            total += labels.numel()

    accuracy = correct / total
    return accuracy

# Calculate accuracy for the test set
test_accuracy = calculate_accuracy(model, test_dataloader)
print(f"Test Accuracy: {test_accuracy:.4f}")

Calculating Accuracy: 100%|██████████| 545/545 [01:13<00:00,  7.44batch/s]

Test Accuracy: 0.8743


In [11]:
def calculate_loss(model, dataloader):
    model.eval()
    total_loss = 0
    total_samples = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating Loss", unit="batch"):
            inputs = batch["input_ids"].to(device)
            outputs = model(inputs, labels=inputs)
            loss = outputs.loss
            total_loss += loss.item() * inputs.size(0)  # Multiply by batch size
            total_samples += inputs.size(0)

    avg_loss = total_loss / total_samples
    return avg_loss

# Calculate loss for the test set
test_loss = calculate_loss(model, test_dataloader)
print(f"Test Loss: {test_loss:.4f}")

Calculating Loss: 100%|██████████| 545/545 [01:17<00:00,  7.05batch/s]

Test Loss: 0.4841


In [12]:
# Access the first transformer block
transformer_block = model.transformer.h[0]

# Extract the combined query, key, and value weights
combined_weights = transformer_block.attn.c_attn.weight.data
print("Combined weights shape:", combined_weights.shape)

# Split the combined weights into query, key, and value weights
embed_dim = model.config.hidden_size
num_heads = model.config.num_attention_heads
head_dim = embed_dim // num_heads

query_weights = combined_weights[:embed_dim, :]
key_weights = combined_weights[embed_dim:2*embed_dim, :]
value_weights = combined_weights[2*embed_dim:, :]

print("Query weights shape:", query_weights.shape)
print("Key weights shape:", key_weights.shape)
print("Value weights shape:", value_weights.shape)

Combined weights shape: torch.Size([768, 2304])
Query weights shape: torch.Size([768, 2304])
Key weights shape: torch.Size([0, 2304])
Value weights shape: torch.Size([0, 2304])


In [13]:
print("Query weights shape:", query_weights.shape)

Query weights shape: torch.Size([768, 2304])


In [14]:
# Reshape the query weights for decomposition
tensor = query_weights.reshape(128, 128, 108)  # Reshape to 3D
print("Reshaped tensor shape:", tensor.shape)

Reshaped tensor shape: torch.Size([128, 128, 108])


In [15]:
import tensorly as tl
from tensorly.decomposition import parafac

# Set the backend to PyTorch
tl.set_backend("pytorch")

# Perform CP decomposition
rank = 5  # Choose the rank for decomposition
factors = parafac(tensor, rank=rank)

# Reconstruct the tensor from factors
reconstructed_tensor = tl.cp_to_tensor(factors)

print("Original tensor shape:", tensor.shape)
print("Reconstructed tensor shape:", reconstructed_tensor.shape)

Original tensor shape: torch.Size([128, 128, 108])
Reconstructed tensor shape: torch.Size([128, 128, 108])


In [16]:
# Reshape the reconstructed tensor back to the original shape
reconstructed_query_weights = reconstructed_tensor.reshape(query_weights.shape)
print("Reconstructed query weights shape:", reconstructed_query_weights.shape)

Reconstructed query weights shape: torch.Size([768, 2304])


In [17]:
# Replace the query weights in the model
combined_weights[:768, :] = reconstructed_query_weights

In [18]:
!pip install tensorboard

In [19]:
import torch

# Clear GPU memory
torch.cuda.empty_cache()

In [20]:
import time
import psutil
import torch
from transformers import Trainer, TrainingArguments

# Define training arguments without evaluation
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=1,  # Reduced batch size
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    report_to="tensorboard",
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=16,  # Increased gradient accumulation steps
    disable_tqdm=False,
    evaluation_strategy="no",  # Disable evaluation entirely
    load_best_model_at_end=False,  # Disabled since no evaluation
)

# Define the Trainer without evaluation metrics/datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    # eval_dataset removed
    # compute_metrics removed
)

# Debug: Print a message before starting training
print("Starting fine-tuning...")

# Measure CPU/GPU usage before training
cpu_start = psutil.cpu_percent(interval=None)
gpu_start = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0

# Start timer
start_time = time.time()

# Start fine-tuning
trainer.train()

# Stop timer
end_time = time.time()

# Measure CPU/GPU usage after training
cpu_end = psutil.cpu_percent(interval=None)
gpu_end = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0

# Calculate metrics
training_time = end_time - start_time
cpu_usage = cpu_end - cpu_start
gpu_usage = (gpu_end - gpu_start) / (1024 ** 2)  # Convert to MB

# Debug: Print a message after training completes
print("Fine-tuning completed!")
print(f"Training Time: {training_time:.2f} seconds")
print(f"CPU Usage: {cpu_usage}%")
print(f"GPU Usage: {gpu_usage:.2f} MB")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting fine-tuning...


Step,Training Loss
100,0.440600
200,0.422700
300,0.403100
400,0.434500
500,0.394100
600,0.409900
700,0.407600
800,0.373300
900,0.427000
1000,0.414200


Fine-tuning completed!
Training Time: 7189.80 seconds
CPU Usage: -0.1999999999999993%
GPU Usage: 851.02 MB
